In statistics, logistic regression, or logit regression, or logit model is a regression model where the dependent variable (DV) is categorical.

Logistic regression is used to describe data and to explain the relationship between one dependent binary variable and one or more nominal, ordinal, interval or ratio-level independent variables.

In [2]:
import tensorflow as tf

In [3]:
#Import MNIST
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot = True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [4]:
#Setting up tensforflow parameters
learning_rate =  0.03 
training_epochs = 25
batch_size = 100

#Building a deeper network
n_hidden_1 = 256 #number of nodes in hidden layer 1
n_hidden_2 = 256 #number of features in hidden layer 2

n_inputs = 784 #MNIST data 28*28=784 for each image
n_classes = 10 #Output can be 1 of the 10 numbers


In [5]:
#TF graph inputs

#mnist images are of shape 28*28 = 784
#We do not know the number of images/ they will be different for train and test thus None, 784
x = tf.placeholder(tf.float32, [None, n_inputs])

#y, i.e. the result can be any number from 0 to 9 in onehot encoded format
y = tf.placeholder(tf.float32, [None, n_classes]) 

In [7]:
#Defining weights and bias
W = {"w1": tf.Variable(tf.random_normal([n_inputs, n_hidden_1])),
     "w2": tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
     "w_": tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
    }

b = {"b1": tf.Variable(tf.random_normal([n_hidden_1])),
     "b2": tf.Variable(tf.random_normal([n_hidden_2])),
     "b_": tf.Variable(tf.random_normal([n_classes]))
    }

In [9]:
#Model construction
def multilayer_perceptron(x, ws, bs):
    #Hidden layer1 with RELU activation
    layer_1 = tf.add(tf.matmul(x,ws["w1"]), bs["b1"])
    output_layer_1 = tf.nn.relu(layer_1)
    #Hidden layer2 with RELU activation
    layer_2 = tf.add(tf.matmul(output_layer_1,ws["w2"]), bs["b2"])
    output_layer_2 = tf.nn.relu(layer_2)
    #Final output layer 
    output = tf.add(tf.matmul(output_layer_2, ws["w_"]), bs["b_"])
    return output

#Model call
pred = multilayer_perceptron(x, W, b)

In [10]:
#Now, every time this model is rerun to train, it will work towards reducing the loss and error
#Loss reduction will be based on cross entropy 1/(1+e**-x)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits= pred, labels= y))

# AdamOptimizer training
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

In [15]:
##Initialize all variables and launch session

init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    
    #Training steps
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        
        #Training will be in batches now
        for i in range(total_batch):
            #pick up one batch at a time
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            _, c = sess.run([optimizer, cost], feed_dict = {x: batch_xs,
                                                  y: batch_ys})
            new_cost = c/total_batch
            #computing intermediary loss
            avg_cost = avg_cost + new_cost
        
        if epoch%5 ==0:
            print "Epoch:", "%02d" % (epoch+1), "Cost:", avg_cost
            
    print "Training finished"  
    
    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # Calculate accuracy for 3000 examples
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print "Accuracy:", accuracy.eval({x: mnist.test.images, y: mnist.test.labels})

Epoch: 01 Cost: 40.5793464544
Epoch: 06 Cost: 0.798111425259
Epoch: 11 Cost: 0.56965863702
Epoch: 16 Cost: 0.475053905411
Epoch: 21 Cost: 0.40898494012
Training finished
Accuracy: 0.8753


This 2 hidden layer model performs better than the regular logistic regression.